<a href="https://colab.research.google.com/github/honeybeat1/klue-sts/blob/main/%EA%B8%B0%EC%97%85%EA%B3%BC%EC%A0%9C3_8%ED%8C%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 0. 설정 및 모듈 임포트

In [ ]:
!pip install transformers datasets wandb

In [ ]:
# 모델 허깅페이스 업로드를 위한 로그인 클라
!huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/token.
        (Deprecated, will be removed in v0.3.0) To login with username and password instead, interrupt with Ctrl+C.
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine

In [ ]:
!pip install hf-lfs
!git config --global user.email "honeybeat1@gmail.com"
!git config --global user.name "lanaChung"

In [ ]:
import transformers
print(transformers.__version__)

4.17.0


In [ ]:
import os
import sys
import pandas as pd
import numpy as np 

import re
import requests
import gc

import torch
import torch.nn as nn

from datasets import Dataset
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

In [ ]:
# GPU 메모리 에러 체크
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [ ]:
# seed
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# device type
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Tesla T4
cuda


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# CUDA Memory Error
print(gc.collect())
torch.cuda.empty_cache()

44


### 1. 데이터셋 불러오기

In [ ]:
# argument setting
task = "sts"
model_checkpoint = "klue/roberta-base" # 사전학습 언어모델
batch_size = 32 

In [ ]:
train_df = pd.read_json('/content/drive/MyDrive/data/klue-sts-v1.1/klue-sts-v1.1_train.json')
dev_df = pd.read_json('/content/drive/MyDrive/data/klue-sts-v1.1/klue-sts-v1.1_dev.json')

### 1-1. 전처리

In [ ]:
train_df.head()

,guid,source,sentence1,sentence2,labels,annotations
0,klue-sts-v1_train_00000,airbnb-rtt,숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.,숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.,"{'label': 3.7, 'real-label': 3.714285714285714...","{'agreement': '0:0:0:2:5:0', 'annotators': ['0..."
1,klue-sts-v1_train_00001,policy-sampled,위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다.,시민들 스스로 자발적인 예방 노력을 한 것은 아산 뿐만이 아니었다.,"{'label': 0.0, 'real-label': 0.0, 'binary-labe...","{'agreement': '5:0:0:0:0:0', 'annotators': ['1..."
2,klue-sts-v1_train_00002,paraKQC-sampled,회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘.,사람들이 주로 네이버 메일을 쓰는 이유를 알려줘,"{'label': 0.30000000000000004, 'real-label': 0...","{'agreement': '4:2:0:0:0:0', 'annotators': ['1..."
3,klue-sts-v1_train_00003,policy-sampled,"긴급 고용안정지원금은 지역고용대응 등 특별지원금, 지자체별 소상공인 지원사업, 취업...","고용보험이 1차 고용안전망이라면, 국민취업지원제도는 2차 고용안전망입니다.","{'label': 0.6000000000000001, 'real-label': 0....","{'agreement': '4:2:1:0:0:0', 'annotators': ['1..."
4,klue-sts-v1_train_00004,airbnb-rtt,"호스트의 답장이 늦으나, 개선될 것으로 보입니다.",호스트 응답이 늦었지만 개선될 것으로 보입니다.,"{'label': 4.7, 'real-label': 4.714285714285714...","{'agreement': '0:0:0:0:2:5', 'annotators': ['1..."


In [ ]:
# 데이터셋 shape 확인
# 약 22배 차이
train_df.shape, dev_df.shape

((11668, 6), (519, 6))

In [ ]:
# 중복행 발견
train_df.duplicated(['sentence1', 'sentence2']).sum()

7

In [ ]:
# sentence1, sentence2 동일한 데이터 발견 
# 하나씩만 남기고 제거
train_df[train_df.duplicated(['sentence1','sentence2'], keep=False)]

,guid,source,sentence1,sentence2,labels,annotations
1514,klue-sts-v1_train_01514,policy-rtt,"제2차 전략회의에서 대전, 경기, 강원, 전남, 제주, 경남 등 6개 시·도지사는 ...","제2차 전략회의에서 대전, 경기, 강원, 전남, 제주, 경남 등 6개 시·도지사가 ...","{'label': 4.7, 'real-label': 4.714285714285714...","{'agreement': '0:0:0:0:2:5', 'annotators': ['0..."
1661,klue-sts-v1_train_01661,airbnb-rtt,택시타고 공항갔을 때 20유로로 15분내에 도착했었어요.,"택시를 타고 공항에 갔을 때, 15분만에 20유로에 도착했습니다.","{'label': 4.7, 'real-label': 4.666666666666667...","{'agreement': '0:0:0:0:2:4', 'annotators': ['1..."
1715,klue-sts-v1_train_01715,airbnb-rtt,택시타고 공항갔을 때 20유로로 15분내에 도착했었어요.,"택시를 타고 공항에 갔을 때, 15분만에 20유로에 도착했습니다.","{'label': 4.7, 'real-label': 4.666666666666667...","{'agreement': '0:0:0:0:2:4', 'annotators': ['1..."
3872,klue-sts-v1_train_03872,policy-rtt,"제2차 전략회의에서 대전, 경기, 강원, 전남, 제주, 경남 등 6개 시·도지사는 ...","제2차 전략회의에서 대전, 경기, 강원, 전남, 제주, 경남 등 6개 시·도지사가 ...","{'label': 4.9, 'real-label': 4.857142857142857...","{'agreement': '0:0:0:0:1:6', 'annotators': ['1..."
5139,klue-sts-v1_train_05139,policy-rtt,지난달 하루 평균 수출액이 전년대비 5.6% 증가한 가운데 자동차는 2017년 11...,"지난 달 평균 일일 수출액이 전년 동월대비 5.6% 증가하면서, 자동차의 수출은 2...","{'label': 4.6, 'real-label': 4.571428571428571...","{'agreement': '0:0:0:0:3:4', 'annotators': ['0..."
5292,klue-sts-v1_train_05292,policy-rtt,"제2차 전략회의에서 대전, 경기, 강원, 전남, 제주, 경남 등 6개 시·도지사는 ...","제2차 전략회의에서 대전, 경기, 강원, 전남, 제주, 경남 등 6개 시·도지사가 ...","{'label': 4.7, 'real-label': 4.714285714285714...","{'agreement': '0:0:0:0:2:5', 'annotators': ['0..."
7045,klue-sts-v1_train_07045,policy-rtt,지난달 하루 평균 수출액이 전년대비 5.6% 증가한 가운데 자동차는 2017년 11...,"지난 달 평균 일일 수출액이 전년 동월대비 5.6% 증가하면서, 자동차의 수출은 2...","{'label': 4.0, 'real-label': 4.0, 'binary-labe...","{'agreement': '0:0:0:1:5:1', 'annotators': ['1..."
10908,klue-sts-v1_train_10908,policy-rtt,지난달 하루 평균 수출액이 전년대비 5.6% 증가한 가운데 자동차는 2017년 11...,"지난 달 평균 일일 수출액이 전년 동월대비 5.6% 증가하면서, 자동차의 수출은 2...","{'label': 4.0, 'real-label': 4.0, 'binary-labe...","{'agreement': '0:0:0:1:5:1', 'annotators': ['1..."
10939,klue-sts-v1_train_10939,policy-rtt,지난달 하루 평균 수출액이 전년대비 5.6% 증가한 가운데 자동차는 2017년 11...,"지난 달 평균 일일 수출액이 전년 동월대비 5.6% 증가하면서, 자동차의 수출은 2...","{'label': 4.6, 'real-label': 4.571428571428571...","{'agreement': '0:0:0:0:3:4', 'annotators': ['0..."
11112,klue-sts-v1_train_11112,policy-rtt,"제2차 전략회의에서 대전, 경기, 강원, 전남, 제주, 경남 등 6개 시·도지사는 ...","제2차 전략회의에서 대전, 경기, 강원, 전남, 제주, 경남 등 6개 시·도지사가 ...","{'label': 4.9, 'real-label': 4.857142857142857...","{'agreement': '0:0:0:0:1:6', 'annotators': ['1..."


In [ ]:
train_df = train_df.drop_duplicates(['sentence1','sentence2'], keep='first', ignore_index=True)

In [ ]:
# 확인
train_df.duplicated(['sentence1', 'sentence2']).sum()

0

In [ ]:
# 확인
dev_df.duplicated(['sentence1','sentence2']).sum()

0

In [ ]:
# 특수문자만 제거하고, 영어, 숫자는 살려보겠습니다
def cleansing(text):
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ·!』\\‘〈〉|\(\)\[\]\<\>`\'…》《]','', text)
    return text

In [ ]:
train_df['cleansed'] = train_df['sentence1'].apply(cleansing)
train_df['cleansed2'] = train_df['sentence2'].apply(cleansing)
dev_df['cleansed'] = dev_df['sentence1'].apply(cleansing)
dev_df['cleansed2'] = dev_df['sentence2'].apply(cleansing)

### Dataset

In [ ]:
params = ['guid','cleansed','cleansed2','labels']

In [ ]:
dataset = Dataset.from_pandas(train_df[params])
test_dataset = Dataset.from_pandas(dev_df[params])

In [ ]:
dataset, test_dataset

(Dataset({
     features: ['guid', 'cleansed', 'cleansed2', 'labels'],
     num_rows: 11661
 }), Dataset({
     features: ['guid', 'cleansed', 'cleansed2', 'labels'],
     num_rows: 519
 }))

In [ ]:
dataset = dataset.rename_column('cleansed','sentencel')
dataset = dataset.rename_column('cleansed2','sentence2')
test_dataset = test_dataset.rename_column('cleansed','sentencel')
test_dataset = test_dataset.rename_column('cleansed2','sentence2')

In [ ]:
dataset

Dataset({
    features: ['guid', 'sentencel', 'sentence2', 'labels'],
    num_rows: 11661
})

In [ ]:
# 어떤 metric으로 계산할지 지정하기
# pearsonr, f1
metric_name = 'f1'

In [ ]:
dataset = dataset.flatten()

if metric_name == 'f1':
  # binary-label을 label로 하여 계산한다
  dataset = dataset.rename_column('labels.binary-label','label')
  dataset = dataset.remove_columns(['labels.real-label','labels.label'])
else:
  # real-label을 label로 하여 계산한다
  dataset = dataset.rename_column('labels.real-label','label')
  dataset = dataset.remove_columns(['labels.binary-label','labels.label'])

In [ ]:
dataset

Dataset({
    features: ['guid', 'sentencel', 'sentence2', 'label'],
    num_rows: 11661
})

### 2. 토큰화

In [ ]:
# klue/roberta-base 모델의 토크나이저 
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/734k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [ ]:
# train을 train/val로 나누고, val을 score용 test로 쓴다. 
# 9:1로 train/val 나누기
train_dataset = dataset.train_test_split(test_size=0.1, shuffle=True, seed=42)

In [ ]:
train_dt = train_dataset['train']
val_dt = train_dataset['test']
val_dt

Dataset({
    features: ['guid', 'sentencel', 'sentence2', 'label'],
    num_rows: 1167
})

In [ ]:
len(train_dt), len(val_dt)

(10494, 1167)

In [ ]:
max_length=512

### tokenizer

In [ ]:
# examples['sentence1']이 자꾸 에러나서, 직접 인덱싱해서 가져오기.
# 이상하게 sentence1을 복사해오면 또 된다..;
def preprocess_function(examples):
    return tokenizer(examples[train_dt.column_names[2]], examples["sentence2"],
                     truncation=True, max_length=512, padding=True)

encoded_train = train_dt.map(preprocess_function)
encoded_val = val_dt.map(preprocess_function)

  0%|          | 0/10494 [00:00<?, ?ex/s]

  0%|          | 0/1167 [00:00<?, ?ex/s]

In [ ]:
encoded_train

Dataset({
    features: ['guid', 'sentencel', 'sentence2', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 10494
})

### 2. Fine-tuning

In [ ]:
# pretrain 모델의 체크포인트를 다운로드하여 STS Task 학습

num_labels = 2 if metric_name == 'f1' else 1
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,
                                                           num_labels=num_labels)

loading configuration file https://huggingface.co/klue/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a96469ca2a238496d435a0e9e202f261119c146a0326444b6d68ae1adc35e04f.85b0b02ba2a483f3adb8a60ab70dbd875768fcd5e6cdb21a593c6e02fdffac3a
Model config RobertaConfig {
  "_name_or_path": "klue/roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertTokenizer",
  "transformers_version": "4.17.0",
  "type_vocab_size": 1,
  "use_cache": true,


In [ ]:
model_name = model_checkpoint.split("/")[-1]
output_dir = os.path.join("test-klue", "sts")
logging_dir = os.path.join(output_dir, 'logs')
args = TrainingArguments(
    # checkpoint
    output_dir=output_dir,
    overwrite_output_dir=True,

    # Model Save & Load
    save_strategy = "epoch", # 'steps'
    load_best_model_at_end=True,
    # save_steps = 500,


    # Dataset
    num_train_epochs=5, # 10
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=8,
    
    # Optimizer
    learning_rate=6e-5, # 5e-5
    weight_decay=0.01,  # 0
    #warmup_steps=200,

    # Regularization
    max_grad_norm = 1.0,
    #label_smoothing_factor=0.1,


    # Evaluation 
    metric_for_best_model='eval_' + metric_name,
    evaluation_strategy = "epoch",
    eval_accumulation_steps=16,

    # HuggingFace Hub Upload
    push_to_hub=True,
    push_to_hub_model_id=f"{model_name}-finetuned-{task}-{metric_name}_",

    # Logging
    logging_dir=logging_dir,
    report_to='wandb',

    # Randomness
    seed=42,

    # Mixed-Precision Training
    fp16=True,
    fp16_opt_level='01',
)

PyTorch: setting up devices
/usr/local/lib/python3.7/dist-packages/transformers/training_args.py:952: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case rurupang/roberta-base-finetuned-sts-f1_).
  FutureWarning,


In [ ]:
from datasets import load_metric
metric_ = load_metric(metric_name)

In [ ]:
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  if metric_name == 'f1':
    predictions = np.argmax(predictions, axis=1)
  else:
    predictions = predictions[:, 0]
  
  result = metric_.compute(predictions=predictions, references=labels)
  return result

In [ ]:
from transformers import DataCollatorWithPadding

# Dynamic padding 
data_collator = DataCollatorWithPadding(tokenizer)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_train,
    eval_dataset=encoded_val,
    data_collator=data_collator, #dynamic padding
    compute_metrics=compute_metrics,
)

/content/test-klue/sts is already a clone of https://huggingface.co/rurupang/roberta-base-finetuned-sts-f1_. Make sure you pull the latest changes with `repo.git_pull()`.
Using amp half precision backend


In [ ]:
# wandb로 기록
import wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [ ]:
id = wandb.util.generate_id()
print(id)

11v78aun


In [ ]:
wandb.init(project='klue-sts-roberta',
           entity='honeybeat1',
           id=id,
           )

eval/f1,▁
eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁
train/global_step,▁
eval/f1,0.72317
eval/loss,0.56716
eval/runtime,2.5949
eval/samples_per_second,449.726


In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence2, sentencel, guid. If sentence2, sentencel, guid are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 10494
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1640


Epoch,Training Loss,Validation Loss,F1
1,No log,0.553589,0.761978
2,0.453600,0.511738,0.778982
3,0.453600,0.610890,0.775908
4,0.320800,0.598075,0.797639
5,0.209700,0.598082,0.797639


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence2, sentencel, guid. If sentence2, sentencel, guid are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1167
  Batch size = 8
Saving model checkpoint to test-klue/sts/checkpoint-328
Configuration saved in test-klue/sts/checkpoint-328/config.json
Model weights saved in test-klue/sts/checkpoint-328/pytorch_model.bin
Several commits (2) will be pushed upstream.
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence2, sentencel, guid. If sentence2, sentencel, guid are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1167
  Batch size = 8
Saving 

TrainOutput(global_step=1640, training_loss=0.3168209587655416, metrics={'train_runtime': 1146.2072, 'train_samples_per_second': 45.777, 'train_steps_per_second': 1.431, 'total_flos': 2163119236096920.0, 'train_loss': 0.3168209587655416, 'epoch': 5.0})

In [ ]:
wandb.finish()

eval/f1,▁▄▄██
eval/loss,▄▁█▇▇
eval/runtime,▁█▁▁▂
eval/samples_per_second,█▁██▇
eval/steps_per_second,█▁██▇
train/epoch,▁▂▃▅▅▆▇██
train/global_step,▁▂▃▅▅▆▇██
train/learning_rate,█▄▁
train/loss,█▄▁
train/total_flos,▁
train/train_loss,▁


In [ ]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence1, guid, sentence2. If sentence1, guid, sentence2 are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1167
  Batch size = 8


{'epoch': 10.0,
 'eval_accuracy': 0.778063410454156,
 'eval_loss': 0.5574014186859131,
 'eval_runtime': 3.8486,
 'eval_samples_per_second': 303.223,
 'eval_steps_per_second': 37.935}

In [ ]:
# 허깅페이스에 저장
trainer.push_to_hub()

### Predict

In [ ]:
test_dataset

Dataset({
    features: ['guid', 'sentence1', 'sentence2', 'label'],
    num_rows: 519
})

In [ ]:
encoded_test = test_dataset.map(preprocess_function)

Loading cached processed dataset at /root/.cache/huggingface/datasets/klue/sts/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e/cache-5c78b5c3a7212060.arrow


In [ ]:
result = trainer.predict(encoded_test)
predicts = np.argmax(result[0],axis=1)

The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence1, guid, sentence2. If sentence1, guid, sentence2 are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 519
  Batch size = 8


In [ ]:
result[2]

{'test_accuracy': 0.7225433526011561,
 'test_loss': 0.6520352959632874,
 'test_runtime': 1.822,
 'test_samples_per_second': 284.853,
 'test_steps_per_second': 35.675}

### dev_set_score 

In [ ]:
dev_set_score = pd.read_csv('정다현 - dev_set_score.csv')
dev_set_score.head()

,guid,true_real_label,true_binary_label,predict_real_label,predict_binary_label
0,klue-sts-v1_dev_00000,4.857143,1,NaN,NaN
1,klue-sts-v1_dev_00001,1.428571,0,NaN,NaN
2,klue-sts-v1_dev_00002,1.285714,0,NaN,NaN
3,klue-sts-v1_dev_00003,3.714286,1,NaN,NaN
4,klue-sts-v1_dev_00004,2.500000,0,NaN,NaN


In [ ]:
len(dev_set_score), len(result[0])

(519, 519)

In [ ]:
# predicted한 real label dev_set_score 파일에 넣기
dev_set_score['predict_real_label'] = result[0] #pearson으로 돌렸던 output
dev_set_score.head()

,guid,true_real_label,true_binary_label,predict_real_label,predict_binary_label
0,klue-sts-v1_dev_00000,4.857143,1,4.980749,NaN
1,klue-sts-v1_dev_00001,1.428571,0,2.145419,NaN
2,klue-sts-v1_dev_00002,1.285714,0,1.488101,NaN
3,klue-sts-v1_dev_00003,3.714286,1,4.164298,NaN
4,klue-sts-v1_dev_00004,2.500000,0,3.016868,NaN


In [ ]:
#일단 저장
dev_set_score.to_csv('기업과제3_8팀_dev_set_score.csv', index=False)

In [ ]:
pwd

'/content'

In [ ]:
cd drive/MyDrive/data

/content/drive/MyDrive/data


In [ ]:
dev_set_score1 = pd.read_csv('기업과제3_8팀_dev_set_score.csv')
dev_set_score1

,guid,true_real_label,true_binary_label,predict_real_label,predict_binary_label
0,klue-sts-v1_dev_00000,4.857143,1,4.980749,NaN
1,klue-sts-v1_dev_00001,1.428571,0,2.145419,NaN
2,klue-sts-v1_dev_00002,1.285714,0,1.488101,NaN
3,klue-sts-v1_dev_00003,3.714286,1,4.164298,NaN
4,klue-sts-v1_dev_00004,2.500000,0,3.016868,NaN
...,...,...,...,...,...
514,klue-sts-v1_dev_00514,2.200000,0,3.660266,NaN
515,klue-sts-v1_dev_00515,2.833333,0,3.013463,NaN
516,klue-sts-v1_dev_00516,0.333333,0,1.506227,NaN
517,klue-sts-v1_dev_00517,0.333333,0,0.609422,NaN


In [ ]:
dev_set_score1['predict_binary_label'] = predicts
dev_set_score1

,guid,true_real_label,true_binary_label,predict_real_label,predict_binary_label
0,klue-sts-v1_dev_00000,4.857143,1,4.980749,1
1,klue-sts-v1_dev_00001,1.428571,0,2.145419,0
2,klue-sts-v1_dev_00002,1.285714,0,1.488101,0
3,klue-sts-v1_dev_00003,3.714286,1,4.164298,1
4,klue-sts-v1_dev_00004,2.500000,0,3.016868,1
...,...,...,...,...,...
514,klue-sts-v1_dev_00514,2.200000,0,3.660266,1
515,klue-sts-v1_dev_00515,2.833333,0,3.013463,0
516,klue-sts-v1_dev_00516,0.333333,0,1.506227,0
517,klue-sts-v1_dev_00517,0.333333,0,0.609422,0


In [ ]:
#저장
dev_set_score1.to_csv('기업과제3_8팀_dev_set_score.csv', index=False)

In [ ]:
# tokenizer push
tokenizer.push_to_hub('rurupang/roberta-base-finetuned-sts')

/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:1004: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,
Cloning https://huggingface.co/rurupang/roberta-base-finetuned-sts into local empty directory.


Download file pytorch_model.bin:   0%|          | 8.00k/422M [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 2.98k/2.98k [00:00<?, ?B/s]

Clean file training_args.bin:  34%|###3      | 1.00k/2.98k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/422M [00:00<?, ?B/s]

tokenizer config file saved in rurupang/roberta-base-finetuned-sts/tokenizer_config.json
Special tokens file saved in rurupang/roberta-base-finetuned-sts/special_tokens_map.json
remote: tput: No value for $TERM and no -T specified        
remote: tput: No value for $TERM and no -T specified
remote: tput: No value for $TERM and no -T specified
remote: tput: No value for $TERM and no -T specified        
To https://huggingface.co/rurupang/roberta-base-finetuned-sts
   ca10106..498ae30  main -> main



'https://huggingface.co/rurupang/roberta-base-finetuned-sts/commit/498ae30cb7b551e9288592b4e11a0eb203eb2840'

### Hyperparameter Search

In [ ]:
!pip install optuna

In [ ]:
model_name = model_checkpoint.split("/")[-1]
output_dir = os.path.join("test-klue", "sts-hps")
logging_dir = os.path.join(output_dir, 'logs')

args = TrainingArguments(
    # checkpoint
    output_dir=output_dir,
    # overwrite_output_dir=True,

    # Model Save & Load
    save_strategy = "epoch", # 'steps'
    load_best_model_at_end=True,
    # save_steps = 500,


    # Dataset
    num_train_epochs=3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    
    # Optimizer
    learning_rate=2e-5, # 5e-5
    weight_decay=0.01,  # 0
    warmup_steps=200,

    # Resularization
    max_grad_norm = 1.0,
    # label_smoothing_factor=0.1,


    # Evaluation 
    metric_for_best_model='eval_pearsonr',
    evaluation_strategy = "epoch",

    # HuggingFace Hub Upload
    push_to_hub=True,
    push_to_hub_model_id=f"{model_name}-finetuned-{task}",

    # Logging
    logging_dir=logging_dir,
    report_to='wandb',

    # Randomness
    seed=42,
        
    # Mixed-Precision Training
    fp16=True,
    fp16_opt_level='01',
)

PyTorch: setting up devices
/usr/local/lib/python3.7/dist-packages/transformers/training_args.py:952: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case rurupang/roberta-base-finetuned-sts).
  FutureWarning,


In [ ]:
def model_init(params):
  print(params)
  if params is not None:
    model_checkpoint.update({'dropout': params['dropout']})
  return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

In [ ]:
trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=encoded_train.shard(index=1, num_shards=10),
    eval_dataset=encoded_val,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

loading configuration file https://huggingface.co/klue/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a96469ca2a238496d435a0e9e202f261119c146a0326444b6d68ae1adc35e04f.85b0b02ba2a483f3adb8a60ab70dbd875768fcd5e6cdb21a593c6e02fdffac3a
Model config RobertaConfig {
  "_name_or_path": "klue/roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertTokenizer",
  "

Download file pytorch_model.bin:   0%|          | 17.1k/422M [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 2.98k/2.98k [00:00<?, ?B/s]

Clean file training_args.bin:  34%|###3      | 1.00k/2.98k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/422M [00:00<?, ?B/s]

Using amp half precision backend


In [ ]:
wandb.login()

True

In [ ]:
id = wandb.util.generate_id()
print(id)

1csuurt0


In [ ]:
wandb.init(project='klue-sts-roberta-base',
           entity='honeybeat1',
           id=id)

In [ ]:
def optuna_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 5e-4, log=True),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 1, 5),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [8, 16, 32, 64, 128]),
        "weight_decay":  trial.suggest_float("weight_decay", 1e-6, 1e-1),
        'warmup_steps': trial.suggest_int('warmup_steps', 0, 500),
    }

In [ ]:
def objective(metrics):
    return metrics["eval_pearsonr"]

In [ ]:
best_run = trainer.hyperparameter_search(n_trials=15, 
                                         direction="maximize", 
                                         hp_space=optuna_hp_space, 
                                         compute_objective=objective)

[I 2022-03-24 01:26:45,821] A new study created in memory with name: no-name-8cb38240-d0ef-4a42-8bf2-22f1cf3b0ae9
Trial:
loading configuration file https://huggingface.co/klue/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a96469ca2a238496d435a0e9e202f261119c146a0326444b6d68ae1adc35e04f.85b0b02ba2a483f3adb8a60ab70dbd875768fcd5e6cdb21a593c6e02fdffac3a
Model config RobertaConfig {
  "_name_or_path": "klue/roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hi

Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Pearsonr
1,No log,3.108283,nan
2,No log,0.366469,0.930078


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: labels.binary-label, labels.label, sentence1, sentence2, guid, source. If labels.binary-label, labels.label, sentence1, sentence2, guid, source are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1167
  Batch size = 32
/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
Saving model checkpoint to test-klue/sts-hps/run-0/checkpoint-33
Configuration saved in test-klue/sts-hps/run-0/checkpoint-33/config.json
Model weights saved in test-klue/sts-hps/run-0/checkpoint-33/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenc

eval/loss,█▁
eval/pearsonr,▁
eval/runtime,█▁
eval/samples_per_second,▁█
eval/steps_per_second,▁█
train/epoch,▁██
train/global_step,▁██
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁


...

The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: labels.binary-label, labels.label, sentence1, sentence2, guid, source. If labels.binary-label, labels.label, sentence1, sentence2, guid, source are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1167
  Batch size = 32
Saving model checkpoint to test-klue/sts-hps/run-13/checkpoint-132
Configuration saved in test-klue/sts-hps/run-13/checkpoint-132/config.json
Model weights saved in test-klue/sts-hps/run-13/checkpoint-132/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: labels.binary-label, labels.label, sentence1, sentence2, guid, source. If labels.binary-label, labels.label, sentence1, sentence2, guid, source are not expected 

eval/loss,▄█▁
eval/pearsonr,▅▁█
eval/runtime,▁██
eval/samples_per_second,█▁▁
eval/steps_per_second,█▁▁
train/epoch,▁▅█
train/global_step,▁▅█
eval/loss,0.36436
eval/pearsonr,0.93094
eval/runtime,3.8495
eval/samples_per_second,303.154


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Pearsonr
1,No log,3.184092,nan


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: labels.binary-label, labels.label, sentence1, sentence2, guid, source. If labels.binary-label, labels.label, sentence1, sentence2, guid, source are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1167
  Batch size = 32
/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/usr/local/lib/python3.7/dist-packages/optuna/pruners/_percentile.py:20: RuntimeWarning: All-NaN slice encountered
  return np.nanmax(values)
[I 2022-03-24 01:42:51,475] Trial 14 pruned. 


In [ ]:
best_run

BestRun(run_id='3', objective=0.9450056084443673, hyperparameters={'learning_rate': 2.3901707640693283e-05, 'num_train_epochs': 5, 'per_device_train_batch_size': 16, 'weight_decay': 0.024030913724965405, 'warmup_steps': 297})

In [ ]:
best_run.hyperparameters

{'learning_rate': 2.3901707640693283e-05,
 'num_train_epochs': 5,
 'per_device_train_batch_size': 16,
 'warmup_steps': 297,
 'weight_decay': 0.024030913724965405}

### 5. Hyperparameter 적용해보기

In [ ]:
model_name = model_checkpoint.split("/")[-1]
output_dir = os.path.join("test-klue", "sts-hps")
logging_dir = os.path.join(output_dir, 'logs')

args = TrainingArguments(
    # checkpoint
    output_dir=output_dir,
    # overwrite_output_dir=True,

    # Model Save & Load
    save_strategy = "epoch", # 'steps'
    load_best_model_at_end=True,
    # save_steps = 500,


    # Dataset
    num_train_epochs=best_run.hyperparameters['num_train_epochs'],
    per_device_train_batch_size=best_run.hyperparameters['per_device_train_batch_size'],
    per_device_eval_batch_size=best_run.hyperparameters['per_device_train_batch_size'],
    
    # Optimizer
    learning_rate=best_run.hyperparameters['learning_rate'],
    weight_decay=best_run.hyperparameters['weight_decay'],
    warmup_steps=best_run.hyperparameters['warmup_steps'],

    # Resularization
    # max_grad_norm = 1.0,
    # label_smoothing_factor=0.1,


    # Evaluation 
    metric_for_best_model='eval_pearsonr',
    evaluation_strategy = "epoch",

    # HuggingFace Hub Upload
    push_to_hub=True,
    push_to_hub_model_id=f"{model_name}-finetuned-{task}",

    # Logging
    logging_dir=logging_dir,
    report_to='wandb',

    # Randomness
    seed=42,
)

PyTorch: setting up devices
/usr/local/lib/python3.7/dist-packages/transformers/training_args.py:952: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case rurupang/roberta-base-finetuned-sts).
  FutureWarning,


In [ ]:
trainer = Trainer(
    model,
    args=args,
    train_dataset=encoded_train,
    eval_dataset=encoded_val,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/content/test-klue/sts-hps is already a clone of https://huggingface.co/rurupang/roberta-base-finetuned-sts. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
import wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [ ]:
id = wandb.util.generate_id()
print(id)
wandb.init(project='klue-sts-roberta',
           entity='honeybeat1',
           id=id,
           )

3uxxo6bh


eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁
train/global_step,▁
eval/loss,3.18409
eval/pearsonr,nan
eval/runtime,3.7583
eval/samples_per_second,310.512
eval/steps_per_second,9.845


In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: labels.binary-label, labels.label, sentence1, sentence2, guid, source. If labels.binary-label, labels.label, sentence1, sentence2, guid, source are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 10501
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3285
Automatic Weights & Biases logging enabled, to 

Epoch,Training Loss,Validation Loss,Pearsonr
1,0.064200,0.306440,0.947460
2,0.087600,0.272878,0.951530
3,0.074000,0.232053,0.950816
4,0.042600,0.217846,0.953703
5,0.031400,0.198718,0.955253


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: labels.binary-label, labels.label, sentence1, sentence2, guid, source. If labels.binary-label, labels.label, sentence1, sentence2, guid, source are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1167
  Batch size = 16
Saving model checkpoint to test-klue/sts-hps/checkpoint-657
Configuration saved in test-klue/sts-hps/checkpoint-657/config.json
Model weights saved in test-klue/sts-hps/checkpoint-657/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: labels.binary-label, labels.label, sentence1, sentence2, guid, source. If labels.binary-label, labels.label, sentence1, sentence2, guid, source are not expected by `RobertaForSequenc

TrainOutput(global_step=3285, training_loss=0.05662812635053057, metrics={'train_runtime': 640.9491, 'train_samples_per_second': 81.918, 'train_steps_per_second': 5.125, 'total_flos': 2087820408960618.0, 'train_loss': 0.05662812635053057, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: labels.binary-label, labels.label, sentence1, sentence2, guid, source. If labels.binary-label, labels.label, sentence1, sentence2, guid, source are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1167
  Batch size = 16


{'epoch': 5.0,
 'eval_loss': 0.1987176239490509,
 'eval_pearsonr': 0.9552527336501904,
 'eval_runtime': 3.3925,
 'eval_samples_per_second': 343.993,
 'eval_steps_per_second': 21.518}

In [ ]:
wandb.finish()

eval/loss,█▆▃▂▁▁
eval/pearsonr,▁▅▄▇██
eval/runtime,▃▂▅▁▃█
eval/samples_per_second,▆▇▄█▆▁
eval/steps_per_second,▆▇▄█▆▁
train/epoch,▁▁▂▃▄▅▅▆▆▇███
train/global_step,▁▁▂▃▄▅▅▆▆▇███
train/learning_rate,█▇▅▄▂▁
train/loss,▅█▆▄▂▁
train/total_flos,▁
train/train_loss,▁


In [ ]:
result = trainer.predict(encoded_test)
result[2]

The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: labels.binary-label, labels.label, sentence1, sentence2, guid, source. If labels.binary-label, labels.label, sentence1, sentence2, guid, source are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 519
  Batch size = 16


{'test_loss': 0.35473135113716125,
 'test_pearsonr': 0.8905772854306188,
 'test_runtime': 1.5465,
 'test_samples_per_second': 335.597,
 'test_steps_per_second': 21.339}

In [ ]:
test_dataset['label'][:5]

[4.857142857142857,
 1.428571428571429,
 1.285714285714286,
 3.714285714285714,
 2.5]

In [ ]:
result[0][0]

array([4.9272723], dtype=float32)

In [ ]:
len(encoded_test)

519